<a href="https://colab.research.google.com/github/bori00/BrainTumorClassification/blob/bori_dev/BrainTumorClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brain Tumor Classification

In [1]:
# Imports
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

import urllib
from urllib import request

import time

In [3]:
# Reproducability
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

In [4]:
tumor_types = ['glioma_tumor', 'meningioma_tumor']

In [5]:
folder_ids = {
    "train": '1HK5JxUCzDAI812XOHEYZPFvVHuKGdxDg',
    "test": '1b3iKJeldP-0zTy6GL2wvytHhhI6WEe9_'
}

In [18]:
# importing zip files from fgoogle drive

folder_ids = {
    "train": '1HK5JxUCzDAI812XOHEYZPFvVHuKGdxDg',
    "test": '1b3iKJeldP-0zTy6GL2wvytHhhI6WEe9_'
}

for folder_type in ['train', 'test']:
    # drive_folder_url = "https://drive.google.com/u/2/uc?id={}&export=download".format(folder_ids[folder_type])
    # drive_folder_url = "https://docs.google.com/uc?export=download&id={}&confirm=t".format(folder_ids[folder_type])

    drive_folder_url = "https://doc-10-bc-docs.googleusercontent.com/docs/securesc/qtq9pa03792qmock0mr06urnpi13hppk/avcfr921na7j8uq8cn35ckfbukcqrut5/1651741650000/18367490022258197177/18367490022258197177/1HK5JxUCzDAI812XOHEYZPFvVHuKGdxDg?e=download&ax=ACxEAsb5t7hsgc2CWxSqb_WH8JBIa2Tey2aBL6UFHyUiK19Z7aWDq3UenNHSJaH-_nGpOxRoN2J5hD52kKcoia8DmMdq7lyIh_YrvFnnTu3q78XrtzaM4HYQd1OUSZMzRXAjV7ED0bGuTUEbw1v7OVt3uGkE72oTxA4g_qsz133n5PJ4UUKv99TiiqbGsnafBAoFBvR5eQSwelMkcaKVYEzG_iG5PnOviFwrx41dGAGpAOjSl6i9dP6nwnv3LtQT8cBsyycychYCy3m8OyerCzsErMVnFqiXESMNmwBmPu5cI9sMpZE_6sFRvyWCB4Fq1EAXRswiJ9EnLQch5Np8dhtTyH2-sKX_hsdaaCH_FYLj517f0kQzNn81TeVYKM7GO9GirezM-5OipmW-yTKfcHepS_YJA6XQqXIcYIXMkKWcxStEzdpl4c8hSXvdoCHPW9RGRBdYfAXoznz3FZKiMOXbIn7CIcK-1MurqwVQwvITnW9rkrEWproCGO_sJPo2x8OCN4gpmLZRfSc9a_qVPDossincUD61TBpQPnYo5Ct5Gl6DzplKWH481sJxEj3TizZWoHYz1DCArMkykyfiz7atA6rDM6AZLctKxVa2NEDbgu8J0fZSevgdMk7K_vk7dXrmTKUkPVMUAmUZ23-IIqb7MSPqMqWxyKaz9Dmfg7i4oNlvhfTcH48bbDdygPYDUCM_7uB2KwV-FRBsf81cElWuOPCd5XE0T4hlSRCHgbIMlW2wQHRf1bo&authuser=0&nonce=cv3iejgcj27mq&user=18367490022258197177&hash=0aeeibv4b7qvgqks2ipgboqt27lmpp3c"
    print(drive_folder_url)

    file_name = '{}.zip'.format(folder_type)

    urllib.request.urlretrieve(drive_folder_url, file_name)

    # req =  request.Request(drive_folder_url, method='POST')
    # resp = request.urlopen(req)

    print('Download completed!', folder_type)

    time.sleep(3)

https://doc-10-bc-docs.googleusercontent.com/docs/securesc/qtq9pa03792qmock0mr06urnpi13hppk/avcfr921na7j8uq8cn35ckfbukcqrut5/1651741650000/18367490022258197177/18367490022258197177/1HK5JxUCzDAI812XOHEYZPFvVHuKGdxDg?e=download&ax=ACxEAsb5t7hsgc2CWxSqb_WH8JBIa2Tey2aBL6UFHyUiK19Z7aWDq3UenNHSJaH-_nGpOxRoN2J5hD52kKcoia8DmMdq7lyIh_YrvFnnTu3q78XrtzaM4HYQd1OUSZMzRXAjV7ED0bGuTUEbw1v7OVt3uGkE72oTxA4g_qsz133n5PJ4UUKv99TiiqbGsnafBAoFBvR5eQSwelMkcaKVYEzG_iG5PnOviFwrx41dGAGpAOjSl6i9dP6nwnv3LtQT8cBsyycychYCy3m8OyerCzsErMVnFqiXESMNmwBmPu5cI9sMpZE_6sFRvyWCB4Fq1EAXRswiJ9EnLQch5Np8dhtTyH2-sKX_hsdaaCH_FYLj517f0kQzNn81TeVYKM7GO9GirezM-5OipmW-yTKfcHepS_YJA6XQqXIcYIXMkKWcxStEzdpl4c8hSXvdoCHPW9RGRBdYfAXoznz3FZKiMOXbIn7CIcK-1MurqwVQwvITnW9rkrEWproCGO_sJPo2x8OCN4gpmLZRfSc9a_qVPDossincUD61TBpQPnYo5Ct5Gl6DzplKWH481sJxEj3TizZWoHYz1DCArMkykyfiz7atA6rDM6AZLctKxVa2NEDbgu8J0fZSevgdMk7K_vk7dXrmTKUkPVMUAmUZ23-IIqb7MSPqMqWxyKaz9Dmfg7i4oNlvhfTcH48bbDdygPYDUCM_7uB2KwV-FRBsf81cElWuOPCd5XE0T4hlSRCHgbIMlW2wQHRf1bo&authuser=0&

In [9]:
file_id = '1b3iKJeldP-0zTy6GL2wvytHhhI6WEe9_'
request = drive_service.files().get_media(fileId=file_id)
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download {}".format(int(status.progress() * 100)))

NameError: ignored

In [ ]:
!unzip 'Training' -d 'Training'

Archive:  Training
checkdir:  cannot create extraction directory: Training
           File exists


In [17]:
import zipfile
zip_ref = zipfile.ZipFile("train.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

BadZipFile: ignored